In [1]:
import numpy as np
import math

### Utility functions

In [2]:
def is_symmetric(A):
    return np.all(A-A.T == 0)

In [3]:
def is_positive_definite(A):
    return np.all(np.linalg.eigvals(A) > 0)

In [4]:
def extended_matrix(A,B):
    return np.append(A, B, axis=1)

In [5]:
def is_singular(A):
    return np.isclose(0, np.linalg.det(A))

### Substitution

In [6]:
def forward_substitution(L, Y):
    n = L.shape[0]
    
    X = np.zeros((n,1))
    
    for i in range(n):
        X[i,0] = Y[i,0] / L[i,i] 
        
        for j in range(i):
            X[i,0] -= L[i,j] * X[j,0] / L[i,i]
            
    return X

In [7]:
def backward_substitution(U, Y):
    n = U.shape[0]
    
    X = np.zeros((n,1))
    
    for i in range(n-1, -1, -1):
        X[i,0] = Y[i,0] / U[i,i]
        
        for j in range(i + 1, n):
            X[i,0] -= U[i,j] * X[j,0] / U[i,i]
            
    return X

### Gauss

In [8]:
def gauss(A, B):
    if is_singular(A):
        raise ValueException("Cannot solve equation for singular matrix")
        
    U = extended_matrix(A, B)
        
    n = U.shape[0]
    
    factor = np.max(np.absolute(A), axis = 1)
    for i in range(n):
        for k in range(n+1):
            U[i,k] /= factor[i]
        
    for i in range(n):
        
        row = i
        for j in range(i+1, n):
            if abs(U[j,i]) > abs(U[row,i]):
                row = j
        U[[i,row]] = U[[row, i]]
        
        for j in range(i + 1, n):
            factor = U[j,i] / U[i,i]
            
            for k in range(n + 1):
                U[j,k] -= U[i,k] * factor
    
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            factor = U[j,i] / U[i,i]
            
            for k in range(n + 1):
                U[j,k] -= U[i,k] * factor
    
    for i in range(n):
        U[i,n] /= U[i,i]
        U[i,i] /= U[i,i]
    
    return U[:,n:n+1]

### Doolittle

In [9]:
def doolittle_decomposition(A):
    n = A.shape[0]
    
    L = np.eye(n)
    U = np.zeros((n,n))
    
    for i in range(n):
        
        # row
        for j in range(i, n):
            U[i,j] = A[i,j]
            
            for k in range(i):
                U[i,j] -= L[i,k] * U[k,j]
        
        # col
        for j in range(i + 1, n):
            L[j,i] = A[j,i]
            
            for k in range(i):
                L[j,i] -= L[j,k] * U[k,i]
                
            L[j,i] /= U[i,i]
    
    return L, U

In [10]:
def doolittle(A, Y):
    L,U = doolittle_decomposition(A)
    Z = forward_substitution(L, Y)
    return backward_substitution(U, Z)

### Crout

In [11]:
def crout_decomposition(A):
    n = A.shape[0]
    
    L = np.zeros((n,n))
    U = np.eye(n)
    
    for i in range(n):
        
        # col
        for j in range(i, n):
            L[j,i] = A[j,i]
            
            for k in range(i):
                L[j,i] -= L[j,k] * U[k,i]
        
        # row
        for j in range(i + 1, n):
            U[i,j] = A[i,j]
            
            for k in range(i):
                U[i,j] -= L[i,k] * U[k,j]
                
            U[i,j] /= L[i,i]
    
    return L, U

In [12]:
def crout(A, Y):
    L,U = crout_decomposition(A)
    Z = forward_substitution(L, Y)
    return backward_substitution(U, Z)

### Cholesky

In [13]:
def cholesky_decomposition(A):
    if not is_symmetric(A):
        raise ValueError("Matrix has to be symmetric to use Cholesky decomposition")
    if not is_positive_definite(A):
        raise ValueError("Matrix has to be positive definite to use Cholesky decomposition")
    
    n = A.shape[0]
    
    L = np.zeros((n,n))
    
    for j in range(n):
        for i in range(j, n):
            L[i,j] = A[i,j]
            
            for k in range(j):
                L[i,j] -= L[i,k] * L[j,k]
            
            if j == i:
                L[i,j] = math.sqrt(L[i,j])
            else:
                L[i,j] /= L[j,j]
    
    return L, L.transpose()

In [14]:
def cholesky(A, Y):
    L,U = cholesky_decomposition(A)
    Z = forward_substitution(L, Y)
    return backward_substitution(U, Z)

---

### Manual test

In [15]:
A = np.array([[4, 12, -16],[12, 37, -43],[-16, -43, 98]]).astype("float32")
Y = np.array([[4],[10],[-6]]).astype("float32")

In [16]:
gauss(A, Y)

array([[ 49.22222  ],
       [-13.1111145],
       [  2.22222  ]], dtype=float32)

In [17]:
doolittle(A, Y)

array([[ 49.22222222],
       [-13.11111111],
       [  2.22222222]])

In [18]:
crout(A, Y)

array([[ 49.22222222],
       [-13.11111111],
       [  2.22222222]])

In [19]:
cholesky(A, Y)

array([[ 49.22222222],
       [-13.11111111],
       [  2.22222222]])

In [20]:
np.linalg.solve(A, Y)

array([[ 49.22222  ],
       [-13.111111 ],
       [  2.2222223]], dtype=float32)

---

### Timing

In [21]:
def test(n):
    print("Test n =", n)
    
    A = np.random.rand(n,n)
    Y = np.random.rand(n,1)
    
    test_gauss      = %timeit -o -r 3 -n 1 gauss(A, Y)
    test_doolittle  = %timeit -o -r 3 -n 1 doolittle(A, Y)
    test_crout      = %timeit -o -r 3 -n 1 crout(A, Y)
    test_np         = %timeit -o -r 3 -n 1 np.linalg.solve(A, Y)
    
    return test_gauss, test_doolittle, test_crout, test_np

In [22]:
results = {n: tuple(map(lambda t: np.average(t.timings), test(n))) for n in range(10, 301, 10)}

Test n = 10
1.95 ms ± 42.3 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
956 µs ± 109 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
901 µs ± 105 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
The slowest run took 206.75 times longer than the fastest. This could mean that an intermediate result is being cached.
8.22 ms ± 11.4 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
Test n = 20
10.4 ms ± 2.74 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
4.33 ms ± 574 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
4.82 ms ± 1.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
153 µs ± 66.7 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)
Test n = 30
41.3 ms ± 5.57 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
12.3 ms ± 1.81 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
9.79 ms ± 2.18 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
The slowest run took 4.24 times longer than the fastest. This could mean that an intermediat

6.37 s ± 22.1 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
2.79 s ± 120 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
2.7 s ± 21.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
The slowest run took 4.85 times longer than the fastest. This could mean that an intermediate result is being cached.
4.53 ms ± 2.55 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
Test n = 260
7.42 s ± 100 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
3.06 s ± 27.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
3.13 s ± 118 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
3.2 ms ± 1.49 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
Test n = 270
8.28 s ± 81.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
3.49 s ± 111 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
4.84 s ± 966 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
5.57 ms ± 2.35 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
Test n = 280
11.5 s ± 973 ms per loop (mean 

In [23]:
headers = ['n','Gauss','Doolittle','Crout','NumPy']

with open('results.csv', 'w') as file:
    file.write(','.join(headers)+'\n')
    for n,res in results.items():
        file.write('{},{},{},{},{}\n'.format(n, *res))